In [23]:
import pymongo
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint

In [24]:
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Backend Final Project-credential.json', scope)
client = gspread.authorize(creds)

sheet = client.open('I-House Mastersheet').get_worksheet(0)

allData = sheet.get_all_records()
for item in allData:
    pprint(item)

{'Dinner: Please check all the items that you would like in your meal box.': 'Broccoli '
                                                                             '(V/GF), '
                                                                             'Korean '
                                                                             'pork '
                                                                             'chops, '
                                                                             'Jerk '
                                                                             'chicken '
                                                                             '(GF)',
 'Do you have any other notes / specifications?': 'test',
 'Email Address': 'maxmir@berkeley.edu',
 'Lunch: Please check all the items that you would like in your meal box.': 'Broccoli '
                                                                            '(V/GF), '
                                

In [25]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["meal_order_db"]
mycol = mydb["meal_order"]

In [26]:
for record in allData:
    

    meal_count = 0
    lunch_menu = {}
    dinner_menu = {}
    menu = [lunch_menu, dinner_menu]
    
    # remove key and value with '.' and create a new key without '.' with the same value
    for key, value in record.items():
#         if "Options:" in key:
#             del record[key]       
#             new_key = key[:8] + " " + str(meal_count+1)
#             stripped = key[9:]
#             menu_options = stripped.split(",")
#             for i in menu_options:
#                 menu[meal_count][i] = False
#             print(menu[meal_count])
#             record[new_key] = menu[meal_count]
#             meal_count+=1
        
        
        if "." in key:
            del record[key]
            new_key = key.replace(".", "")
            record[new_key] = value           
    record["Is_completed"] = False
    mycol.insert_one(record)

In [27]:
for item in mydb.meal_order.find():
    pprint(item)

{'Dinner: Please check all the items that you would like in your meal box': 'Broccoli '
                                                                            '(V/GF), '
                                                                            'Korean '
                                                                            'pork '
                                                                            'chops, '
                                                                            'Jerk '
                                                                            'chicken '
                                                                            '(GF)',
 'Do you have any other notes / specifications?': 'test',
 'Email Address': 'maxmir@berkeley.edu',
 'Lunch: Please check all the items that you would like in your meal box': 'Broccoli '
                                                                           '(V/GF), '
                                          

 'Email Address': 'maxmir@berkeley.edu',
 'Is_completed': False,
 'Lunch: Please check all the items that you would like in your meal box': 'Broccoli '
                                                                           '(V/GF), '
                                                                           'Korean '
                                                                           'pork '
                                                                           'chops, '
                                                                           'Jerk '
                                                                           'chicken '
                                                                           '(GF)',
 'Name': 'Test2',
 'Numbers on the back of your Cal1Card': 987,
 'Phone Number (to contact if needed)': 1111111111,
 'Please mark ALL allergies/sensitivities (or list additional below)': 'Dairy',
 'Please mark all dietary restrictions ': 'Vegetarian',
 'Ple